# TITLE

"ESTIMACION SOBRE LAS NECESIDADES DEL AREA DE SERVICIOS SOCIALES DEL AYUNTAMIENTO DE MADRID EN RELACION CON LAS URGENCIAS DE SERVICIOS" 

## SUBJECT:

"REALIZAR UNA CLASIFICACIÓN EN FUNCIÓN DE LAS CARACTERÍSTICAS MÁS IMPORTANTES DE LOS DATASETS DEL AYTO. DE MADRID DE LAS URGENCIAS ATENDIDAS EN LOS CENTROS DE ATENCIÓN PRIMARIA PARA TIPIFICAR EL TIPO DE AYUDA DE URGENCIA QUE NECESITARÁN.

CON ESTO PRETENDEMOS QUE AL LLEGAR UNA SOLICITUD DE URGENCIA A UN CENTRO PUEDAN IR PREPARANDO LOS RECURSOS HUMANOS Y NO HUMANOS PARA AGILIZAR TODOS LOS PROCESOS REQUERIDOS EN CADA TIPO DE AYUDA."

"HACER UNA ESTIMACION GLOBAL Y PARCIAL DE LAS NECESIDADES A ATENDER EN EL AREA DE SERVICIOS SOCIALES DEL AYUNTAMIENTO DE MADRID"

Se tomarán los datos de los últimos años y se realizará un estudio tomando como referencia *julio de 2021*, ya que no hay datos posteriores.

## HYPOTHESIS:

**hipótesis 1** = "Es posible establecer un patrón de usuarios que van a solicitar alguna urgencia para poder atender mejor y más eficientemente modo a éstos."

**hipótesis 2** = "Es necesario establecer algún tipo de estudio temporal que permita optimizar los recursos del Area de Servicios Sociales?"

## 1.- Data Collection:

La fuente principal de todos los datos obtenidos ha sido a través de las páginas del ayuntamiento de Madrid:

MUNIMADRID y PORTAL DE DATOS ABIERTOS DEL AYUNTAMIENTO DE MADRID)

URL_urgencias_atendidas = <https://datos.madrid.es/portal/site/egob/menuitem.c05c1f754a33a9fbe4b2e4b284f1a5a0/?vgnextoid=0b006dace9578610VgnVCM1000001d4a900aRCRD&vgnextchannel=374512b9ace9f310VgnVCM100000171f5a0aRCRD&vgnextfmt=default>

Las urgencias que se atienden en el Ayuntamiento son 5 y están clasificadas del siguiente modo:

  >Malos tratos:                            --      Código: 10
  >
  >Abandono                                 --      Código: 20
  >
  >Alojamiento/Vivienda                     --      Código: 30
  >
  >Desprotección social problema salud      --      Código: 40
  >
  >Desprotección socio-familiar             --      Código: 50




In [1]:
'''
Para este projecto usaremos las siguientes librerías:
'''
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import pickle 
'''
También nos harán falta las siguientes librerías de sklearn
'''
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,GradientBoostingClassifier,BaggingClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.inspection import permutation_importance
from sklearn.metrics import accuracy_score,precision_score,recall_score,roc_auc_score,f1_score,confusion_matrix

## 2.- Data Extraction from different Datasets and storage in data\Outcomes

Vamos a tomar los datos de los años __desde el 2018 al 2021__.

Los datos de este último año no están completos. Solo hay hasta el mes de Julio.

In [2]:
df_2021=pd.read_csv("E:\Bootcamp_22\Javier\Repositorios\Machine_Learning\data\Incomes\servicios_sociales_urgencias_2021.csv",sep=';',encoding='ISO-8859-1',index_col='Secuencia')   
#Este  dataset solo contine información hasta julio de 2021
df_2020=pd.read_csv("E:\Bootcamp_22\Javier\Repositorios\Machine_Learning\data\Incomes\servicios_sociales_urgencias_2020.csv",sep=';',encoding='utf-8',index_col='Secuencia')

df_2019=pd.read_csv("E:\Bootcamp_22\Javier\Repositorios\Machine_Learning\data\Incomes\servicios_sociales_urgencias_2019.csv",sep=';',encoding='ISO-8859-1',index_col='Secuencia')

df_2018=pd.read_csv("E:\Bootcamp_22\Javier\Repositorios\Machine_Learning\data\Incomes\servicios_sociales_urgencias_2018.csv",sep=';',encoding='ISO-8859-1',index_col='Secuencia')

## 3.- Featuring Engineering

En los diferentes Datasets que hemos almacenado vamos a realizar un reagrupamiento en la columna de **Tramo Edad** para que los menores de 18 años estén todos juntos, los mayores de 65 años también los agruparemos conjutamente y cada tramo de edad vaya de 10 años en 10 años.

En la columna de **Sexo** también se observa que hay unos pocos registros vacios, que los rellenaremos con la moda.

Vamos a extraer de la columna **Fecha Cita** el año del registro para valorarlo independientemente.

El mayor problema que nos encontramos en los Datasets es la presencia de NaNs en las columnas de **Tipo Supuesto Urgente** y **Codigo Tipo Supuesto Urgente**, que son nuestra ***variable target***.

El total de NaNs existentes es de 2184, que procederemos a rellenar Dataset por Dataset valorando los datos de media y mediana de cada uno y observando los datos antes de la modificación y después  para ver si ha habido algún tipo de variación.

Tras estas operaciones vamos a realizar un mapeo en las columnas *Sexo*, *Nacionalidad* y *Tramo Edad*, con el siguiente codigo tipo (en este caso para Tramo Edad):

In [ ]:
#lista_rango_edad=df['Tramo Edad'].unique()                 
#lista_rango_edad.sort()
#mivalor = [ x for x in range(len(lista_rango_edad))]            
#lista_rango_edad=list(lista_rango_edad)                       

#Age_range_code = {k: v for k, v in zip(lista_rango_edad, mivalor)} 
#Age_range_code
#df['Age_range_code']= df['Tramo Edad'].map(Age_range_code)

Los valores así dados son: 

19-29: 0, 30-39: 1, 40-49: 2, 50-59: 3, 60-64: 4, <=18: 5, >=65: 6

Cabría la posibilidad de haber realizado este mapeo de un modo distinto para *Tramo Edad* y dejar ordenado las clases de esta variable, pero al no tener darle un valor concreto a los mismos los de hemos dejado así.

En este sentido, se podría haber dado la media de las edades de cada tramo, para los menores de 18 años, haber dado un valor relativo al porcentaje de cada tramo y lo mismo a los mayores de 65 años.

Realizado todos estos procesos, hemos juntado los Datasets a utilizar para proceder al análisis y modelado.

Vamos a hacer nota aqui que se observa que el Dataset de 2020 es mucho mayor que el resto de Datasets obtenidos.

Así mismo se observa al realizar un estudio de la **variable target del 2020** esta se encuentra desbalanceada y esto provocará un desbalanceo general en el DataFrame que vamos a construir a partir de estos.

En este sentido hemos realizado primero un modelado obviando el desbalanceo y paralelamente hemos realizado una ingeniería de datos para intentar balancear el 2020 y realizar un modelado con otro DataFrame.

Los procesos de sampleado, data engineering y modelado se encuentras en los notebooks:

- Notebook_SAMPLING_3, Notebook_SAMPLING_4, Notebook_SAMPLING_5, Notebook_SAMPLING_6_modeling

Los resultados de estos procesos se han anotado aparte y están en el apartado correspondiente.

## 4 - Exploratory Data Analysis

Aqui vamos a representar los gráficos más oportunos para el estudio preliminar de los datos que hemos obtenido y haremos una breve descripción de los mismos.

Estos gráficos están realizados a *seaborn* y *tableau*.

- #### 4.1- Seaborn

- ####  Vamos a presentar aqui la matriz de correlación obtenida:

![Aqui se observa el mapa de correlacion de las variables obtenidas](./resources/img/mapa_correlacion.png)

Se observa que hay cierta correlación entre los parámetros de **year**, **Nationality_code**,**Código Distrito Centro**.

También se observa que no hay aparente correlación con los parámetros **Código Tipo Atención**,**Age_range_code** y **Sex_code**.

**year** también parece tener correlación con otros parámetros como **Nationality_code** o **Código Distrito centro**.

**Nationality code** y **Age_range_code** parece que también tiene una cierta correlación.

En este sentido cabe preguntarse si las correlaciones de **year** con el resto de los datos es provocada por el gran número de registros existentes en el 2020.

-  ####  Vamos a presentar diagramas de *pairplot* de las diferentes variables entre sí y con la variable target:

![Aqui se observa las relaciones entre las diversas variables](./resources/img/pairplot.png)

En este **pairplot** se ve claramente que los valores no son continuos, sino discretos, por lo que habrá que hacer __modelos de clasificación__.

También se observa que datos de cada variable han dado más resultados en relación con la *variable target* (Código Tipo Supuesto Urgente).

En este sentido cabe destacar los altos valores en el segundo trimestre del 2020, debido al estado de excepción provocado por el COVID-19, por el cierre de muchos negocios y que es la causa principal de muchas de las demandas en los Centros de Atención Primaria en el país, desde esta fecha en adelante.

También se ve como la clase **50** de la variable targert es la que más datos registra. 

- #### 6.2- Aqui se observan diversas gráficas para un estudio preliminar que hemos realizado con **Tableau** referente a los diferentes parametros en relación con la variable objetivo.

![Aqui se observa como evolucionan los distintos casos en el periodo de estudio](./resources/img/Tipologia_casos.png)

Vemos aqui que los datos que se tienen de cada clase de la **variable target** son constantes en el tiempo, salvo un pico que hay en el año 2020 desde el mes de marzo en adelante. 

Esto es debido a la variable externa *COVID-19* que provoco una gran necesidad de servicios sociales.

Se ve como este pico es sobre todo en las clases 40 y 50, siendo peculiar en esta última que en otros periodos de los que tenemos datos no es una clase predominante.

Con esta información que tenemos podemos dar respuesta a la ***segunda hipótesis*** planteada: 

**"Es necesario establecer algún tipo de estudio temporal que permita optimizar los recursos del Area de Servicios Sociales?"**

No, no es necesario, ya que los datos demuestran que los datos datos son estacionarios.

En este sentido, si podemos tomar los datos promedio mes a mes para tener preparados los recursos básicos para atender las urgencias.

**Se puede realizar algún estudio de tipo temporal para establecer qué recursos harán falta en otro momento crítico como en el 2020?**

En este sentido solo disponemos registros de esta  eventualidad. Sería interesante disponer de datos de otras eventualidades ocurridas en el pasado para poder hacer estimaciones de lapso de duración e intensidad en los picos.

![Aqui se observa como evolucionan los distintos casos en el periodo de estudio](./resources/img/Tipologia_casos_por_distrito.png)

Hemos dejado en este gráfico unicamente los valores de las clases que más destacaban en el gráfico anterior y hemos hecho un estudio sobre su incidencia en los distintos distritos de Madrid.

Se ve que la incidencia es sobre todo en el distrito de *Tetuan*, despúes en *Chamberí* y luego en *Fuencarral-El Pardo* (esto choca con los datos que manejamos en mi anterior estudio de EDA sobre la situación de los Centros de Atención Primaria).

#### En los siguientes gráficos vamos a centrarnos más en la tipología de los usuarios de urgencia

![Aqui se observa como evoluciona en el tiempo las atenciones de urgencia](./resources/img/Edades.png)


Aqui vemos la relación entre los rango de edad y las distintas clases de la *variable target*.

Prácticamente la mitad de las urgencias se reparten entre los mayores de 65 años y el rango de edad de *30-39*. 

Cabe destacar en este rango de edad y en general la segunda clase corresponde a *Alojamiento/Vivienda*.

![Aqui se observa como evolucionan los distintos casos en el periodo de estudio](./resources/img/Nacionalidad.png)

Aqui podemos ver que por norma general los usuarios de urgencias han sido de nacionalidad española.

También se observa que debido a la pandemia los usuarios de nacionalidad extranjera aumentaron drásticamente en los trimestres del 2020, para volver luego a normalizar la norma que los españoles acuden en un ratio aproximado de 2:1.

![Aqui se observa como evolucionan los distintos casos en el periodo de estudio](./resources/img/Sexo.png)

Podemos ver que los usuarios habituales son mujeres, independientemente del periodo del año que observemos.

También que los motivos  habituales de las urgencias han sido *Alojamiento/Vivienda* y *Desprotención Social problemas de salud*.

## 5.- Checking models

Como hemos indicado para proceeder a dar respuesta a la *hipótesis 1*, vamos a realizar modelados clasificatorios de las **features** más relevantes que hemos visto en en **Análisis Exploratorio de datos**, salvo la variable *year* que hemos visto que su relación con la variable *target* es debida a una causa externa en el tiempo (estado de excepción y efectos derivados de este).

Los módelos que vamos a utilizar para esto son:

> AdaBoostClassifier de DecisionTreeClassifier  
>
> BaggingClassifier de DecisionTreeClassifier  
>
> DecisionTreeClassifier
>
> GradientBoostingClassifier
>
> Logistic Regresion
>
> K-Nearest Neighbours          
>
> Random Forest
>
> XGBoost Random Forest          (pendiente de realizar)
>
> GradientBoostingClassifier

Como hemos indicado al estar el DataFrame desbalanceado y ser multiclase (5 posibles resultados), nos hemos decidido por realizar métricas dónde se evalue los aciertos de cada clase. Es por esto que tomamos la métrica de *f-1 score* para evaluar los resultados obtenidos.

Y en este sentido tomaremos la métrica *f-1 score macro* para poder contabilizar todas las clases existentes.

Dado que hemos realizado cuatro tipos distintos de modelado, que a continuación detallaremos, vamos a mostrar aqui los resultados obtenidos en los mismos.

- Hemos realizado un modelado sin realizar ninguna regularización en los módelos: (Notebook_5_simple_models)

![Errores de los modelos](./resources/img/Error_simple_models.png)

En esta imagen se puede ver las metricas de cada módelo.

Se puede observar que el mejor módelo sería un **Boost GradientBoosting Regressor**.

También se ve que los módelos **KNearest Neighbors** y **Regresión Logística** dan metricas más bajas, por lo que no seguiremos con ellos en los siguientes pasos.

- Hemos realizado un modelado realizando un Grid Search para regularizar los módelos: (Notebook_6_GS_models)

![Errores de los modelos_regularizados](./resources/img/Error_GS_models.png)

En este sentido se puede ver que los resultados obtenidos apenas varían con los resultados de los módelos iniciales.

- Hemos realizado un modelado regularizando diversos parámetros de manera manual: (Notebook_7_handmade_models)

![Errores de los modelos_artesanos](./resources/img/Error_handmade_models.png)

Se observa que los resultados mejoran en decimas de tanto por ciento sobre los resultados de módelos iniciales.

- Hemos realizado un modelado sobre el DataFrame resampleado sin realizar ninguna regularización: (Notebook_SAMPLING_6_modeling)

![Errores de los modelos_con resammple](./resources/img/Error_resampled_models.png)

Se puede ver que el resampleado realizado no ha dado un resultado satisfactorio.

Los resultados obtenidos fueron más de 10 puntos inferior al resto de los modelados.

### 6.- Best model

El **mejor modelado** que hemos obtenido de las diferentes pruebas realizadas ha sido entonces:

**BaggingClassifier**

Usando *Árboles de Decisión* como base_estimator.

En este sentido la profundidad de los Árboles de Decisisión ha sido de 9.

El Bagging ha utilizado los 4 Features para realizar sus estimaciones, con una muestra máxima de 500 y con 250 n_estimator.

![Aqui viene el modelo como tal](./resources/img/my_best_model.png)

Los datos de las metricas obtenidas tampoco es que hayan sido especialmente más altas al resto.

En este sentido observando la matriz de confusión se puede ver que las clases menos predominantes dan más errores que en las clases menos dominantes.

## 7.- Importance of the features

Tomando como referencia el mejor módelo que es capaz de predecir , vamos a hacer un pequeño estudio sobre qué *Features* son más importantes a la hora de realizar las previsiones (Notebook_8_Features_Importance).

En este sentido hemos realizado 2 estudios:

- Feature Importance: se evalua los valores de cada *Feature* en cada estimación que hacen los Árboles de decisión, tomando su media.

Los datos de la importancia tomada de cada Feature es:

![Aqui se observa los valores de cada Feature en la evaluación del modelo](./resources/img/Feature_importance_DataFrame.png)

![Aqui se observa los valores de cada Feature en la evaluación del modelo](./resources/img/Feature_importance.png)

El módelo establece un **mayor valor** al *Código de Distrito Centro*, esto es al lugar de residencia de la persona usuaria y un **menor valor** al *Sexo* de la persona usuaria.

- Permutation Importance: donde ha realizado un estudio sobre como cambia el resultado de cada predicción al modificar los datos de cada Feature.
Se basa en el hecho que una variable que apenas influya en la predicción no importará si cambia o no los valores. Mientras que una variable que sí influya en la predicción su modificación afectará en la predicción final.

Los datos de la importancia tomada de cada Feature es:

![Aqui se observa los valores de cada Feature en la evaluación del modelo](./resources/img/Permutation_importance_DataFrame.png)

![Aqui se observa los valores de cada Feature en la evaluación del modelo](./resources/img/Permutation_importance.png)

Con este nuevo módelo de análisis de features se ve aún con más nitidez lo visto en el anterior.

Aqui se puede observar que el feature *SEXO* apenas afecta a las decisiones del módelo para dar un resultado u otro.

## 8 CONCLUSIÓN:

### Hipótesis 1:

Con este módelo de ensamblaje (**BaggingClassifier**), podemos determinar qué requisitos puede solicitar un usuario tipo en cada distrito, en función de su sexo, nacionalidad o edad y disponer de una mejor optimización de los recursos de cada centro de atención primaria.



### Hipótesis 2:

Como hemos indicado anteriormente, dada la estacionalidad de los datos que disponemos no es necesario implantar un módelo de tiempo, ya que la tipología de los casos es muy estable en el tiempo.

Si puede ser interesante intentar recabar más información referente a situaciones anómalas como la ocurrida en el 2020, para poder hacer estimaciones oportunas de momentos de crisis.

Vamos a realizar unas pruebas para ver como funciona el módelo, podemos generar varios 'usuarios_posibles' para hacerlos pasar por el módelo y dar predicciones sobre qué tipo de ayuda solicitará.  

In [ ]:
#GENERAMOS USUARIOS y PASAMOS POR EL MODELO